In [2]:
import os
import cv2
import keras
import numpy as np
import pandas as pd
from keras import Sequential
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16

In [3]:
TRAIN_LABELS_FILE = "/home/katarzyna/Desktop/datasets/dresses/train/labels.txt"
VAL_LABELS_FILE = "/home/katarzyna/Desktop/datasets/dresses/val/labels.txt"
TEST_LABELS_FILE = "/home/katarzyna/Desktop/datasets/dresses/test/labels.txt"

In [4]:
df = pd.read_csv('/home/katarzyna/Desktop/datasets/dresses/train/labels.txt', delimiter=' ',
                 names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor',
                        'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")

In [5]:
def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv('/home/katarzyna/Desktop/datasets/dresses/train/labels.txt', delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", class_mode="raw", y_col=['beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], target_size=(128, 128), batch_size=20)

In [6]:
def show_dataset(flow):
    for img, l in flow:
        # wypisz etykietę
        print(l[0])
        # konwersja obrazu do wyświetlenia
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # wyświetl obraz
        cv2.imshow("image", img)
        # poczekaj na klawisz
        cv2.waitKey()

In [7]:
def create_model(gen):
    model = VGG16()
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [8]:
def get_callbacks():
    callbacks =[]
    mc=keras.callbacks.ModelCheckpoint("weights1.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    callbacks.append(mc)
    return callbacks

In [9]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE)
    val_gen = get_generator(VAL_LABELS_FILE)
    callbacks = get_callbacks()
    # stwórz model
    model = create_model(train_gen)
    #model.load_weights('weights1.hdf5')
    #model.summary()
    # trening :)
    model.fit_generator(train_gen, epochs=30, steps_per_epoch=train_gen.n // train_gen.batch_size)
                        #validation_data = val_gen, validation_steps = val_gen.n // val_gen.batch_size) 
                        #callbacks=callbacks)

In [ ]:
main()

Found 3514 validated image filenames.
Found 0 validated image filenames.


/home/katarzyna/.local/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 3514 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  .format(n_invalid, x_col)


369131520/553467096 [===================>..........] - ETA: 23s